In [1]:
import ee 
import folium

In [2]:
def add_ee_layer(self, ee_image_object, vis_params, name):
    map_id_dict = ee.Image(ee_image_object).getMapId(vis_params)
    folium.raster_layers.TileLayer(
        tiles=map_id_dict['tile_fetcher'].url_format,
        attr='Map Data &copy; <a href="https://earthengine.google.com/">Google Earth Engine</a>',
        name=name,
        overlay=True,
        control=True
    ).add_to(self)

# Add the custom method to the folium Map class.
folium.Map.add_ee_layer = add_ee_layer


In [3]:
# Initialize the Earth Engine module.
ee.Initialize()

# Define the start and end date.
START = '2021-04-02'
END = '2023-04-03'

# Define the region of interest.
region = ee.Geometry.Point(14.0, 37.5)

# Filter the collections by region and date.
colFilter = ee.Filter.And(
    ee.Filter.bounds(region),
    ee.Filter.date(START, END))

dwCol = ee.ImageCollection('GOOGLE/DYNAMICWORLD/V1').filter(colFilter)
s2Col = ee.ImageCollection('COPERNICUS/S2').filter(colFilter)

# Ensure collections are not empty
dwCol_size = dwCol.size().getInfo()
s2Col_size = s2Col.size().getInfo()

if dwCol_size == 0 or s2Col_size == 0:
    raise ValueError("One or both of the collections are empty. Please adjust your filters.")

# Join corresponding DW and S2 images (by system:index).
DwS2Col = ee.Join.saveFirst('s2_img').apply(dwCol, s2Col,
    ee.Filter.equals(leftField='system:index', rightField='system:index'))

# Extract an example DW image and its source S2 image.
dwImage = ee.Image(DwS2Col.first())
s2Image = ee.Image(dwImage.get('s2_img'))

# Define list pairs of DW LULC label and color.
CLASS_NAMES = [
    'water', 'trees', 'grass', 'flooded_vegetation', 'crops',
    'shrub_and_scrub', 'built', 'bare', 'snow_and_ice']

VIS_PALETTE = [
    '419bdf', '397d49', '88b053', '7a87c6', 'e49635', 'dfc35a', 'c4281b',
    'a59b8f', 'b39fe1']

# Create an RGB image of the label (most likely class) on [0, 1].
dwRgb = dwImage.select('label').visualize(min=0, max=8, palette=VIS_PALETTE).divide(255)

# Get the most likely class probability.
top1Prob = dwImage.select(CLASS_NAMES).reduce(ee.Reducer.max())

# Create a hillshade of the most likely class probability on [0, 1].
top1ProbHillshade = ee.Terrain.hillshade(top1Prob.multiply(100)).divide(255)

# Combine the RGB image with the hillshade.
dwRgbHillshade = dwRgb.multiply(top1ProbHillshade)

# Define the add_ee_layer function to add Earth Engine layers to folium.
def add_ee_layer(self, ee_image_object, vis_params, name):
    map_id_dict = ee.Image(ee_image_object).getMapId(vis_params)
    folium.raster_layers.TileLayer(
        tiles=map_id_dict['tile_fetcher'].url_format,
        attr='Map Data &copy; <a href="https://earthengine.google.com/">Google Earth Engine</a>',
        name=name,
        overlay=True,
        control=True
    ).add_to(self)

# Add the custom method to the folium Map class.
folium.Map.add_ee_layer = add_ee_layer

# Create a folium map.
Map = folium.Map(location=[37.5, 14.0], zoom_start=8)

# Add the Sentinel-2 image layer.
s2_vis_params = {
    'min': 0,
    'max': 3000,
    'bands': ['B4', 'B3', 'B2']
}
Map.add_ee_layer(s2Image, s2_vis_params, 'Sentinel-2 L1C')

# Add the Dynamic World visualization layer.
dw_vis_params = {
    'min': 0,
    'max': 0.65
}
Map.add_ee_layer(dwRgbHillshade, dw_vis_params, 'Dynamic World V1 - label hillshade')

# Add a layer control panel to the map.
Map.add_child(folium.LayerControl())

# Display the map.
Map


c:\Users\Davide\miniconda3\Lib\site-packages\ee\deprecation.py:204: DeprecationWarning: 

Attention required for COPERNICUS/S2! You are using a deprecated asset.
To ensure continued functionality, please update it.
Learn more: https://developers.google.com/earth-engine/datasets/catalog/COPERNICUS_S2

  warnings.warn(warning, category=DeprecationWarning)
